In [1]:
import os
import pandas as pd
import numpy as np
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
import pickle

In [10]:
!curl http://localhost:5000/get-mock-data

[
  {
    "cindex_2_5": 0.5937763347204995,
    "cindex_97_5": 0.83728808649784,
    "mean_cindex": 0.7231834827878094,
    "outcome": "Abdominal aortic aneurysm",
    "predictor": "metscore"
  },
  {
    "cindex_2_5": 0.6011621364591283,
    "cindex_97_5": 0.6639524805051472,
    "mean_cindex": 0.6404126326179058,
    "outcome": "Atrial fibrillation",
    "predictor": "metscore"
  },
  {
    "cindex_2_5": 0.6894794753045052,
    "cindex_97_5": 0.7720368143657839,
    "mean_cindex": 0.7304459757433557,
    "outcome": "Cardiovascular death",
    "predictor": "metscore"
  },
  {
    "cindex_2_5": 0.6344085860270982,
    "cindex_97_5": 0.6882565235436985,
    "mean_cindex": 0.6679428030033854,
    "outcome": "Coronary artery disease",
    "predictor": "metscore"
  },
  {
    "cindex_2_5": 0.6465412112137625,
    "cindex_97_5": 0.7218716868990149,
    "mean_cindex": 0.6887608440890588,
    "outcome": "Heart failure",
    "predictor": "metscore"
  },
  {
    "cindex_2_5": 0.6523753420771823

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 51673  100 51673    0     0  8718k      0 --:--:-- --:--:-- --:--:--  9.8M


# 从这里开始！

主要的Python包是
- from lifelines import CoxPHFitter
- import pickle


# One sample testing

## Load data

In [13]:
mean_var_df = pd.read_csv('mean_var_df.csv') # 这个数据已放到前端
omics_percentiles = pd.read_csv('omics_percentiles.csv') # 下面两个数据已合并成发给你的percentiles.csv
townsend_percentiles = pd.read_csv('townsend_percentiles.csv')

# 模拟用户输入的数据
user_values = {
    'outcome': 'cad',
    'duration': 10,
    'age': 70,
    'sbp': 120,
    'dbp': 80,
    'height': 170,
    'weight': 70,
    'waist_cir': 90,
    'waist_hip_ratio': 0.9,
    'bmi': 24,
    'baso': 0.01,
    'eos': 0.1,
    'hct': 40,
    'hb': 14,
    'lc': 1.8,
    'mc': 0.5,
    'nc': 4,
    'plt': 250,
    'wbc': 6,
    'townsend': 50,
    'prs': 50,
    'metscore': 50,
    'proscore': 50,
    'sex': 'male',
    'ethnicity': 'white',
    'current_smoking': 'yes',
    'daily_drinking': 'yes',
    'healthy_sleep': 'yes',
    'physical_act': 'yes',
    'healthy_diet': 'yes',
    'social_active': 'yes',
    'family_heart_hist': 'yes',
    'family_stroke_hist': 'yes',
    'family_hypt_hist': 'yes',
    'family_diab_hist': 'yes',
    'lipidlower': 'yes',
    'antihypt': 'yes',
    'hypt_hist': 'yes',
    'diab_hist': 'yes'
 }

In [14]:
mean_var_df

,variables,mean,var
0,age,56.387882,67.201623
1,sbp,137.763589,345.696058
2,dbp,82.340530,102.099311
3,height,168.105964,84.705741
4,weight,77.295436,242.057752
5,waist_cir,89.626129,173.301746
6,waist_hip_ratio,0.867457,0.007814
7,bmi,27.277582,22.008426
8,baso,0.033328,0.002451
9,eos,0.171656,0.019255


In [15]:
omics_percentiles

,omics,outcome,p1,p2,p3,p4,p5,p6,p7,p8,...,p91,p92,p93,p94,p95,p96,p97,p98,p99,p100
0,prs,cad,-3.850298,-3.767714,-3.685130,-3.602547,-3.519963,-3.437379,-3.354795,-3.272212,...,3.582243,3.664826,3.747410,3.829994,3.912578,3.995161,4.077745,4.160329,4.242913,4.325497
1,prs,stroke,-3.564642,-3.491842,-3.419042,-3.346241,-3.273441,-3.200641,-3.127840,-3.055040,...,2.987389,3.060189,3.132989,3.205790,3.278590,3.351390,3.424191,3.496991,3.569791,3.642592
2,prs,hf,-3.740060,-3.655570,-3.571080,-3.486590,-3.402100,-3.317610,-3.233120,-3.148631,...,3.864036,3.948526,4.033016,4.117506,4.201995,4.286485,4.370975,4.455465,4.539955,4.624445
3,prs,af,-4.000919,-3.917444,-3.833969,-3.750495,-3.667020,-3.583545,-3.500070,-3.416595,...,3.511818,3.595293,3.678768,3.762243,3.845717,3.929192,4.012667,4.096142,4.179617,4.263092
4,prs,va,-3.195248,-3.122382,-3.049517,-2.976651,-2.903786,-2.830920,-2.758054,-2.685189,...,3.362660,3.435526,3.508391,3.581257,3.654122,3.726988,3.799854,3.872719,3.945585,4.018451
5,prs,pad,-3.808645,-3.725622,-3.642599,-3.559576,-3.476554,-3.393531,-3.310508,-3.227485,...,3.663409,3.746431,3.829454,3.912477,3.995500,4.078523,4.161545,4.244568,4.327591,4.410614
6,prs,aaa,-3.539402,-3.459204,-3.379007,-3.298809,-3.218612,-3.138415,-3.058217,-2.978020,...,3.678364,3.758562,3.838759,3.918956,3.999154,4.079351,4.159549,4.239746,4.319943,4.400141
7,prs,vt,-3.854540,-3.761034,-3.667527,-3.574021,-3.480515,-3.387009,-3.293502,-3.199996,...,4.561025,4.654532,4.748038,4.841544,4.935050,5.028557,5.122063,5.215569,5.309076,5.402582
8,prs,cvd_death,-3.850298,-3.767714,-3.685130,-3.602547,-3.519963,-3.437379,-3.354795,-3.272212,...,3.582243,3.664826,3.747410,3.829994,3.912578,3.995161,4.077745,4.160329,4.242913,4.325497
9,metscore,cad,-7.260664,-7.144432,-7.028199,-6.911967,-6.795734,-6.679502,-6.563270,-6.447037,...,3.200252,3.316485,3.432717,3.548950,3.665182,3.781415,3.897647,4.013879,4.130112,4.246344


In [16]:
townsend_percentiles

,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,...,p91,p92,p93,p94,p95,p96,p97,p98,p99,p100
0,-1.564706,-1.511044,-1.457382,-1.403719,-1.350057,-1.296395,-1.242733,-1.189071,-1.135409,-1.081747,...,3.264885,3.318547,3.37221,3.425872,3.479534,3.533196,3.586858,3.64052,3.694182,3.747844


下面这部分除了percentiles.csv的映射都在前端完成了，参考function collectFormData()

In [17]:
# Assuming the following data frames are available:
# mean_var_df, omics_percentiles, townsend_percentiles, user_values

# Step 1: Standardize continuous variables using mean and var from mean_var_df
def standardize_continuous_variables(user_values, mean_var_df):
    standardized_values = {}
    for variable in ['age', 'sbp', 'dbp', 'height', 'weight', 'waist_cir', 'waist_hip_ratio', 
                     'bmi', 'baso', 'eos', 'hct', 'hb', 'lc', 'mc', 'nc', 'plt', 'wbc']:
        mean = mean_var_df.loc[mean_var_df['variables'] == variable, 'mean'].values[0]
        var = mean_var_df.loc[mean_var_df['variables'] == variable, 'var'].values[0]
        standardized_values[variable] = (user_values[variable] - mean) / np.sqrt(var)
    return standardized_values

# Step 2: Map percentiles for prs, metscore, proscore using omics_percentiles
def map_percentiles(user_values, omics_percentiles):
    outcome = user_values['outcome']
    mapped_values = {}
    for omics_type in ['prs', 'metscore', 'proscore']:
        percentile_index = user_values[omics_type]
        # Filter the omics_percentiles DataFrame by both 'outcome' and 'omics_type'
        relevant_percentiles = omics_percentiles[(omics_percentiles['outcome'] == outcome) & 
                                                 (omics_percentiles['omics'] == omics_type)]
        # Get the bin centers for the given outcome and omics_type
        bin_centers = relevant_percentiles.iloc[0, 2:].values
        # Store the mapped percentile value
        mapped_values[omics_type] = bin_centers[percentile_index - 1]
    return mapped_values

# Step 3: Map Townsend score using townsend_percentiles
def map_townsend(user_values, townsend_percentiles):
    percentile_index = user_values['townsend']
    # Find the closest bin to the townsend score
    bin_centers = townsend_percentiles.iloc[0].values
    return bin_centers[percentile_index - 1]

# Step 4: One-hot encode categorical variables
def one_hot_encode_categorical_variables(user_values):
    # One-hot encode sex (male=1, female=0)
    sex_encoded = {'male_1.0': 1 if user_values['sex'] == 'male' else 0, 
                   'male_1.0': 0 if user_values['sex'] == 'female' else 0}
    
    # One-hot encode ethnicity
    ethnicity_encoded = {'ethnicity_1.0': 0, 'ethnicity_2.0': 0, 'ethnicity_3.0': 0}
    if user_values['ethnicity'] == 'asian':
        ethnicity_encoded['ethnicity_2.0'] = 1
    elif user_values['ethnicity'] == 'black':
        ethnicity_encoded['ethnicity_3.0'] = 1
    elif user_values['ethnicity'] == 'others':
        ethnicity_encoded['ethnicity_1.0'] = 1
    
    # One-hot encode health-related variables (yes = 1, no = 0)
    health_variables = [
        'current_smoking', 'daily_drinking', 'healthy_sleep', 'physical_act', 
        'healthy_diet', 'social_active', 'family_heart_hist', 'family_stroke_hist',
        'family_hypt_hist', 'family_diab_hist', 'diab_hist', 'hypt_hist', 'lipidlower', 'antihypt'
    ]
    
    health_encoded = {}
    for var in health_variables:
        health_encoded[f'{var}_1.0'] = 1 if user_values.get(var, 'no') == 'yes' else 0
    
    return {**sex_encoded, **ethnicity_encoded, **health_encoded}


# Step 5: Combine all features into a final DataFrame
def process_user_input(user_values, mean_var_df, omics_percentiles, townsend_percentiles):
    # Step 1: Standardize continuous variables
    standardized_values = standardize_continuous_variables(user_values, mean_var_df)
    
    # Step 2: Map percentiles for prs, metscore, proscore
    mapped_percentiles = map_percentiles(user_values, omics_percentiles)
    
    # Step 3: Map Townsend score
    mapped_townsend = map_townsend(user_values, townsend_percentiles)
    
    # Step 4: One-hot encode categorical variables (sex, ethnicity)
    categorical_encoded = one_hot_encode_categorical_variables(user_values)
    
    # Combine all values into a single dictionary
    final_values = {**standardized_values, **categorical_encoded, 'townsend': mapped_townsend, **mapped_percentiles}
    
    # Convert to DataFrame
    final_df = pd.DataFrame([final_values])
    return final_df

# Example usage
final_user_input_df = process_user_input(user_values, mean_var_df, omics_percentiles, townsend_percentiles)

final_user_input_df

,age,sbp,dbp,height,weight,waist_cir,waist_hip_ratio,bmi,baso,eos,...,family_hypt_hist_1.0,family_diab_hist_1.0,diab_hist_1.0,hypt_hist_1.0,lipidlower_1.0,antihypt_1.0,townsend,prs,metscore,proscore
0,1.660488,-0.955396,-0.231634,0.205794,-0.468912,0.0284,0.368153,-0.698649,-0.471173,-0.516398,...,1,1,1,1,1,1,1.064738,0.196307,-1.565276,0.608083


变量名需要和下面一样，不然无法输入模型

In [18]:
final_user_input_df.columns

Index(['age', 'sbp', 'dbp', 'height', 'weight', 'waist_cir', 'waist_hip_ratio',
       'bmi', 'baso', 'eos', 'hct', 'hb', 'lc', 'mc', 'nc', 'plt', 'wbc',
       'male_1.0', 'ethnicity_1.0', 'ethnicity_2.0', 'ethnicity_3.0',
       'current_smoking_1.0', 'daily_drinking_1.0', 'healthy_sleep_1.0',
       'physical_act_1.0', 'healthy_diet_1.0', 'social_active_1.0',
       'family_heart_hist_1.0', 'family_stroke_hist_1.0',
       'family_hypt_hist_1.0', 'family_diab_hist_1.0', 'diab_hist_1.0',
       'hypt_hist_1.0', 'lipidlower_1.0', 'antihypt_1.0', 'townsend', 'prs',
       'metscore', 'proscore'],
      dtype='object')

## Load models

疾病简写映射：
- 'cad': 'Coronary artery disease',
- 'stroke': 'Stroke',
- 'hf': 'Heart failure',
- 'af': 'Atrial fibrillation',
- 'va': 'Ventricular arrhythmias',
- 'pad': 'Peripheral artery disease',
- 'aaa': 'Abdominal aortic aneurysm',
- 'vt': 'Venous thromboembolism',
- 'cvd_death': 'Cardiovascular death'

In [20]:
# 根据用户选择的疾病加载模型
def load_cox_model(user_values):
    # Get the outcome from user values
    outcome = user_values['outcome']
    
    # Define the model path
    model_dir = '/home/luoyan/phd_project/MultiomicsCVD/saved/models/CPH'
    
    # Define the model filename based on the outcome
    model_filename = f'cph_{outcome}.pkl'
    
    # Construct the full path to the model
    model_path = os.path.join(model_dir, model_filename)
    
    # Check if the model file exists
    if os.path.exists(model_path):
        # Load the model
        with open(model_path, 'rb') as f:
            model = pickle.load(f)
        return model
    else:
        raise ValueError(f"Model for outcome '{outcome}' not found in {model_dir}.")

model = load_cox_model(user_values)
model.print_summary()

<lifelines.CoxPHFitter: fitted with 19689 total observations, 18225 right-censored observations>
             duration col = 'duration'
                event col = 'event'
                penalizer = 0.03
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 19689
number of events observed = 1464
   partial log-likelihood = -13671.65
         time fit was run = 2025-03-02 08:21:09 UTC

---
                        coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                               
age                     0.12      1.13      0.03            0.07            0.17                1.07                1.19
townsend                0.03      1.03      0.02           -0.02            0.07                0.98                1.07
sbp                     0.07      1.07      0.03            0.02            0.12                1.02                1.12
dbp                    -0.02      0.98      0.03           -0.07            0.03                0.93                1.03
height                 -0.02      0.98      0.03           -0.08            0.03                0.92                1.03
weight                 -0.02      0.98      0.03           -0.09            0.04                0.92                1.05
waist_cir               0.01      1.01      0.03           -0.06            0.07                0.94                1.08
waist_hip_ratio         0.05      1.05      0.03           -0.01            0.10                0.99                1.11
bmi                    -0.01      0.99      0.03           -0.07            0.05                0.93                1.05
baso                   -0.01      0.99      0.02           -0.05            0.04                0.95                1.04
eos                     0.01      1.01      0.02           -0.03            0.05                0.97                1.05
hct                    -0.02      0.98      0.03           -0.08            0.04                0.92                1.04
hb                      0.02      1.02      0.03           -0.04            0.08                0.96                1.09
lc                     -0.02      0.98      0.02           -0.07            0.03                0.93                1.03
mc                      0.04      1.04      0.02            0.00            0.08                1.00                1.09
nc                      0.01      1.01      0.03           -0.04            0.06                0.96                1.07
plt                    -0.01      0.99      0.02           -0.06            0.03                0.95                1.03
wbc                     0.00      1.00      0.03           -0.06            0.07                0.94                1.07
male_1.0                0.17      1.18      0.06            0.05            0.28                1.05                1.33
ethnicity_1.0          -0.11      0.89      0.16           -0.43            0.21                0.65                1.23
ethnicity_2.0           0.42      1.53      0.15            0.13            0.71                1.14                2.04
ethnicity_3.0          -0.14      0.87      0.18           -0.50            0.22                0.61                1.24
current_smoking_1.0    -0.01      0.99      0.07           -0.15            0.13                0.86                1.13
daily_drinking_1.0     -0.06      0.94      0.05           -0.17            0.05                0.85                1.05
healthy_sleep_1.0      -0.14      0.87      0.05           -0.23           -0.04                0.79                0.96
physical_act_1.0       -0.03      0.97      0.06           -0.15            0.09                0.86                1.10
healthy_diet_1.0        0.00      1.00      0.05           -0.09            0.09                0.92                1.09
social_active_1.0      -0.14      0.87      0.07        

In [11]:
disease_prob = 1 - model.predict_survival_function(final_user_input_df, times=10)
disease_prob = disease_prob.values.flatten()
disease_prob

array([0.06944509])